In [ ]:
%pip install -r requirements.txt

In [5]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key= os.getenv("GROQ_API_KEY")
groq_api_key


'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'

In [6]:
from langchain_groq import ChatGroq

model= ChatGroq(model="openai/gpt-oss-120b", api_key=groq_api_key)
model

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 32768, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000025BC2DDE1E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000025BC51CE9C0>, model_name='openai/gpt-oss-120b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, SystemMessage,AIMessage
from langchain_core.output_parsers import StrOutputParser

parser=StrOutputParser()

In [8]:
model.invoke(
    [
        HumanMessage(content="Hi my name is Swajan I am learning Gen AI"),
        AIMessage(content="Hey Swajan! Nice meeting you. Really excited about your journey! Happy learning!"),
        HumanMessage(content="Hey what is my name? what do i do?")    
    ]
)

AIMessage(content='Your name is **Swajan**.  \n\nYou mentioned that you’re learning\u202fGen\u202fAI, so you’re probably:\n\n- Exploring the basics of generative models (like GPT, diffusion models, etc.)  \n- Trying out some hands‑on experiments with prompts, code, or toolkits such as Hugging\u202fFace, LangChain, or OpenAI’s API  \n- Reading up on topics like prompt engineering, model fine‑tuning, safety, and evaluation  \n\nIf you’d like, I can help you:\n\n1. **Pick a next learning step** – e.g., a quick project, a tutorial, or a deeper dive into a specific concept.  \n2. **Find resources** – articles, videos, notebooks, or courses tailored to your current level.  \n3. **Answer questions** – anything from “what’s the difference between a decoder‑only and an encoder‑decoder model?” to “how do I fine‑tune a small language model on my own data?”\n\nJust let me know what you’d like to focus on next!', additional_kwargs={'reasoning_content': 'The user asks: "Hey what is my name? what do 

### Message History

We can use Message History class to wrap our model and make it stateful. This wll kepp track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input.

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str) -> BaseChatMessageHistory:
    
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
        
    return store[session_id]

with_message_history= RunnableWithMessageHistory(model,get_session_history)

In [10]:
config={"configurable":{"session_id":"chat1"}}

In [13]:
response=with_message_history.invoke(
    [
       HumanMessage(content="Hi my name is Swajan I am learning Gen AI"), 
    ],
    config=config
)

In [14]:
response.content

'Hello Swajan! 👋 It’s great to meet you, and welcome to the world of Generative AI. How can I help you on your learning journey? Are you interested in:\n\n- The fundamentals (models, training data, diffusion, transformers, etc.)  \n- Practical hands‑on projects (text generation, image synthesis, code assistants)  \n- Specific tools and frameworks (TensorFlow, PyTorch, Hugging Face, LangChain, Stable Diffusion, etc.)  \n- Ethics, prompt engineering, or deployment strategies  \n\nFeel free to let me know what you’re curious about or any particular question you have, and I’ll do my best to guide you! 🚀'

In [15]:
response=with_message_history.invoke(
    [
       HumanMessage(content="What is my name"), 
    ],
    config=config
)

In [16]:
response.content

'Your name is **Swajan**.'

In [17]:
config={"configurable":{"session_id":"chat2"}}

response=with_message_history.invoke(
    [
       HumanMessage(content="what is my name?"), 
    ],
    config=config
)

In [18]:
response.content

'I don’t actually know your name—unless you’ve shared it with me in this conversation, I don’t have that information. If you’d like me to address you by name, just let me know what you’d prefer!'

### Prompt Templates

Prompt Templates help to turn raw user information inot a format that the LLM can work with. In this case, the raw user input is just a message which we are passing to the LLM. Lets now make that a bit more complicated. First, lets add in a system message with some custom instrcutions. Next we'll add in more input besides the messages.

In [19]:
prompt_template= ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the questions to the best of your capacity"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt_template|model

In [20]:
response=chain.invoke({
    "messages":[HumanMessage(content="Hi my name is Swajan!")],
    
})

response.content

'Hello Swajan! Nice to meet you. How can I assist you today?'

In [21]:
prompt_template= ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the questions to the best of your capacity in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt_template|model

with_message_history=RunnableWithMessageHistory(chain,get_session_history, input_messages_key="messages")
config={
    "configurable":{
        "session_id":"chat5"
    }
}




In [22]:
response= with_message_history.invoke(
    {
        "messages":[HumanMessage(content="Hey my name is Shyam")],
        "language":"Bangla"
    },
    config=config
)

response.content

'হ্যালো শ্যাম! আপনার সাথে কথা বলে ভালো লাগল। আজ আমি কীভাবে আপনার সাহায্য করতে পারি?'

### Managing the Conversation History

trim_messages is a helper to reduce how many messages we are sending to the model. The trimmer allows us to specify how many token we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages.

In [23]:
from langchain_core.messages import SystemMessage,trim_messages


trimmer= trim_messages(max_tokens=100,
                       strategy="last",
                       token_counter=model,
                       include_system= True,
                       allow_partial=False,
                       start_on="human")

messages=[
    SystemMessage(content="You are a good assistant"),
    HumanMessage(content="Hi my name is Swajan"),
    AIMessage(content="Hi Swajan nice to meet you!"),
    HumanMessage(content="nice to meet you too. can you tell me about AI"),
    AIMessage(content="Sure i can help you understand AI. Lets begin"),
    HumanMessage(content="That would be great. I like Vanilla Ice cream"),
    AIMessage(content="yes!")
]

trimmer.invoke(messages)

[SystemMessage(content='You are a good assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi my name is Swajan', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hi Swajan nice to meet you!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='nice to meet you too. can you tell me about AI', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Sure i can help you understand AI. Lets begin', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='That would be great. I like Vanilla Ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [24]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# RunnablePassthrough lets you surgically modify parts of the input 
# while guaranteeing the rest of the data keeps flowing untouched through the chain.

# itemgetter("messages"): Grabs the full list of messages (history + current input).

# | trimmer: Immediately passes that list through the "Bouncer" to shorten it.

# assign(messages=...): Takes the shortened list and overwrites the original messages key in your data bag.

print(RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer))
chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)|prompt_template|model
)

response=chain.invoke({
    "messages": messages + [HumanMessage(content="what ice cream do i like")],
    "language":"English"
})

response.content

mapper={
  messages: RunnableLambda(itemgetter('messages'))
            | RunnableLambda(...)
}


'You mentioned that you like vanilla ice cream. 🍦'

In [23]:
with_message_history= RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config={
    "configurable":
        {
            "session_id":"chat6"
        }
}

response= with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="What is my name")],
        "language":"English"
    },
    config=config
)

response.content

'Your name is Swajan.'

## Vectors and Retrievers

Documents
Vector Stores
Retrievers

### Documents

Langchain implements a Document abstraction, which is intended to represent a unit of text and associated metadata. It has two attributes:

page_content: a string representing the content;
metadata: a dict containing arbitrary metadata;

The metadata attribute can capture information about the source of the document, its relationship to other documents, and other information. Note than an individual Document object often represent a chunk of a larger document.

In [32]:
from langchain_core.documents import Document

documents=[
    Document(
        page_content="I love samosa",
        metadata={"source":"foods-india"}
    ),
    Document(
        page_content="I love cats",
        metadata={"source":"pets-india"}
    ),
    Document(
        page_content="I love dogs",
        metadata={"source":"pets-india"}
    ),
    Document(
        page_content="I love pav-bhaji",
        metadata={"source":"foods-india"}
    ),
    Document(
        page_content="I love football",
        metadata={"source":"sports-india"}
    )
]

In [26]:
print(documents)

[Document(metadata={'source': 'foods-india'}, page_content='I love samosa'), Document(metadata={'source': 'foods-india'}, page_content='I love rosogolla'), Document(metadata={'source': 'foods-india'}, page_content='I love sambar'), Document(metadata={'source': 'foods-india'}, page_content='I love pav-bhaji'), Document(metadata={'source': 'foods-india'}, page_content='I love sev-puri')]


In [27]:
# %pip install --upgrade transformers sentence-transformers torch
%pip install hf_xet

Note: you may need to restart the kernel to use updated packages.


In [28]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [33]:
# Vector Store
from langchain_chroma import Chroma

vector_store= Chroma.from_documents(documents=documents,embedding=embeddings)
vector_store


In [34]:
vector_store.similarity_search("north indian food")

[Document(id='1338a3da-dc53-47fd-bae9-851177d948f1', metadata={'source': 'foods-india'}, page_content='I love pav-bhaji'),
 Document(id='bdbb98f9-5d3f-4516-8d83-6bd7ea180812', metadata={'source': 'foods-india'}, page_content='I love pav-bhaji'),
 Document(id='4aec3cf1-1ab6-4384-9af1-d7d1a00106d6', metadata={'source': 'foods-india'}, page_content='I love samosa'),
 Document(id='ae35c61e-89e3-4a25-bf38-c8eacbf66d08', metadata={'source': 'foods-india'}, page_content='I love samosa')]

In [35]:
vector_store.similarity_search_with_score("cricket")

[(Document(id='4f013d5a-ba0e-4839-988e-e2f4f5cdc7dc', metadata={'source': 'sports-india'}, page_content='I love football'),
  1.2660973072052002),
 (Document(id='1338a3da-dc53-47fd-bae9-851177d948f1', metadata={'source': 'foods-india'}, page_content='I love pav-bhaji'),
  1.539424180984497),
 (Document(id='bdbb98f9-5d3f-4516-8d83-6bd7ea180812', metadata={'source': 'foods-india'}, page_content='I love pav-bhaji'),
  1.539424180984497),
 (Document(id='deaa49f1-19ad-48f2-9f0a-b16710fe315a', metadata={'source': 'pets-india'}, page_content='I love dogs'),
  1.6288678646087646)]

### Retrievers

Langchain VectorStore objects do not subclass Runnable, so cannot immedately be integrated into LCEL chains.

Langchain Retrievers are Runnables, so they implement a standard set of methods( synchronous and asynchronous invoke and batch ops) and are desugned to be incorporated in LCEL chains.

We can create a simple version of this ourselves without subclassing Retriever. If we choose what method we wish to use to retrieve documents, we can create a runnable easily. Below we will build one around the similarity search method.

So a vector store into a retirever so that it can be a runnable

In [36]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever= RunnableLambda(vector_store.similarity_search).bind(k=1)
retriever.batch(["football","cat"])

[[Document(id='4f013d5a-ba0e-4839-988e-e2f4f5cdc7dc', metadata={'source': 'sports-india'}, page_content='I love football')],
 [Document(id='a649b28f-f838-427a-9307-efb531dd8700', metadata={'source': 'pets-india'}, page_content='I love cats')]]

VectorStores implement an as_retriever method that will geerate a Retriever, specifically a VectorStoreRetriever. These retrievers include search_type and search_kwargs attributes that identify what methods of the underlying vector store to call, and how to parameterize them. For instance we can replicate the above with the following:

In [37]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)

retriever.batch(["cat","dog"])

[[Document(id='a649b28f-f838-427a-9307-efb531dd8700', metadata={'source': 'pets-india'}, page_content='I love cats')],
 [Document(id='deaa49f1-19ad-48f2-9f0a-b16710fe315a', metadata={'source': 'pets-india'}, page_content='I love dogs')]]

### Retriever As Chain

In [38]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message= """
Answer thus question using the provided context only.
{question}

Context:
{context}
"""

prompt_template= ChatPromptTemplate.from_messages(
    [
        ("human",message)
    ]
)

rag_chain={"context":retriever, "question":RunnablePassthrough()}|prompt_template|model

response=rag_chain.invoke("tell me about football")
response.content

'The only information provided in the context is the sentence:\n\n**“I love football.”**'